# 출력 파서(output_parser)
    '03_output_parser.ipynb'

- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 구조화 : LLM의 자유 형식 출력을 구조화된 데이터로 변환
- 일관성 : 항상 일관된 출력 형식 -> 후속 처리 용이
- 유연성 : 다양한 출력 형식(json, list, dict 등)

// 내 머릿 속 가설을 검증해보자 자유 형식은 아마도 str, 일관된 출력 형식도 오류 받아서 피드백 반양할듯 쿼리를 다시 요청하거나 변환을 아예 하거나 


## 'PydanticOutputParser'

In [5]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [2]:
email_conversation = """
From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
from langchain_core.prompts import PromptTemplate

# prompt = PromptTemplate.from_template(
#     """ 아래 이메일 내용 중 중요한 것만 추출해
# ---
#     {email_conversation}"""

# )

prompt = PromptTemplate.from_template(
    '아래 이메일 내용 중 중요한 것만 추출해 \n\n---\n\n{email_conversation}'
)


# print(prompt.format(email_conversation=email_conversation))

chain = prompt | llm

print('---출력 파서 없는 요약---')
chain.invoke({'email_conversation': email_conversation}).content




아래 이메일 내용 중 중요한 것만 추출해 

---


From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션



In [ ]:
class EmailSummary(BaseModel):
    person : str = Field(description='매일 보낸 사람')
    email : str = Field(description='보낸 사람 메일 주소')
    subject : str = Field(description='매일 제목')
    summary : str= Field(description='매일 본문 요약')
    date : str = Field(description='매일 언급된 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)

print(parser.get_format_instructions()) # 코드를 읽는게 아니라 지침을 LLM이 읽는 것

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "매일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸 사람 메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "매일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "매일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "매일 언급된 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [10]:
prompt = PromptTemplate.from_template(
    """ 
너는 분석철학의 창조자야 아래 질문에 맞게 답변을 한국어로 만들어줘
질문 : {question
이메일 내용 : {email_conversation}
형식 : {format}}
    """
)

# 프롬프트 변수들 중 일부만 채우기
prompt.partial(format=parser.get_format_instructions())



PromptTemplate(input_variables=['question\n이메일 내용 '], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "매일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸 사람 메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "매일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "매일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "매일 언급된 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject",

In [ ]:
chain = prompt | llm | parser

# res는 객체 
res = chain.invoke( 
    {
        "question" : '이메일 내용 중 중요한 내용을 추출해줘', 
        'email_conversation' : email_conversation
    }
)

res.model_dump_json()

KeyError: "Input to PromptTemplate is missing variables {'question\\n이메일 내용 '}.  Expected: ['question\\n이메일 내용 '] Received: ['question', 'email_conversation']\nNote: if you intended {question\n이메일 내용 } to be part of the string and not a variable, please escape it with double curly braces like: '{{question\n이메일 내용 }}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:
from pprint import pprint
pprint(res.model_dump())

# pprint : 이쁘게 뽑기, print의 print 

# 감정 분석 Agent

In [2]:
from langchain.agents import create_tool_calling_agent,AgentExecutor,create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field 

# 1. pydantic 클래스 (답변 형태)
class Classification(BaseModel):
    sentiment : str = Field(description='글의 감정')
    aggressiveness: int = Field(description='얼마나 공격적인지 [1~10]점으로 리커드 스케일로 판단')
    language : str = Field(description='작성된 글의 언어')
    remarks: str = Field(description='특이사항 요약[30]글자 이내, 동사형으로')

# 2. LLM
llm = ChatOpenAI(model='gpt-4.1-nano',temperature=0)
structured_llm = llm.with_structured_output(Classification) 


# 3. 프롬프트 정의

inp = "주문한 음식에서 머리카락이 나옴, 분노를 금 할 수 없다. 좋은 말로 할 때 환불을 해라"
# structured_llm.invoke(inp)

memory_prompt = f""" 
- 확실하지 않으면 'chat_history'를 모두 사용해야 해 
"""

prompt = ChatPromptTemplate([
    ('system', '너는 텍스트에서 감정, 공격성, 언어, 특이사항을 추출하는 분류기야. 지금까지의 대화내용을 종합해서 한국어로 대답해야해'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])


# 4. 메모리 정의
memory = ConversationBufferMemory(
    return_messages = True,
    memory_key = 'chat_history'
)


# 5. Agent 조립 
agent = create_openai_tools_agent(
    llm = llm,
    tools = [],
    prompt = prompt 
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = [],
    memory = memory,
    verbose=True 
)

from langchain_core.runnables import RunnableLambda 

pipeline = (
    agent_executor
    | RunnableLambda(lambda d: d['output'])
    | structured_llm
)

pipeline.invoke({'input': inp})
# pipline.invoke({'input':inp})



> Entering new AgentExecutor chain...
알겠습니다. 고객님의 분노와 불편함을 이해하며, 정중하게 환불 요청을 전달하겠습니다. 잠시만 기다려 주세요.

> Finished chain.


Classification(sentiment='중립', aggressiveness=2, language='Korean', remarks='고객의 감정을 이해하며 정중하게 환불 요청을 전달하는 내용입니다.')